In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import tensorflow as tf


csv_path = 'structured_data_mac.csv'
df = pd.read_csv(csv_path)
image_paths = df['Image Path'].tolist()
labels = df['Group'].apply(lambda x: 1 if x == 'AD' else 0).tolist()


def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(value).numpy()]))


def serialize_example(image, label):
    feature = {
        'image': _bytes_feature(image),
        'label': _bytes_feature(label),
    }
    example_proto = tf.train.Example(
        features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


def convert_nii_to_tfrecord(nii_paths, labels, tfrecord_file):
    with tf.io.TFRecordWriter(tfrecord_file) as writer:
        for path, label in zip(nii_paths, labels):
            nii_data = nib.load(path).get_fdata().astype(np.float32)
            nii_data = np.reshape(nii_data, (96, 96, 96, 1))  # Ensure shape
            tf_example = serialize_example(
                nii_data, np.array([label], dtype=np.int32))
            writer.write(tf_example)


convert_nii_to_tfrecord(image_paths, labels, "dataset.tfrecord")

In [ ]:
def _parse_function(proto):

    keys_to_features = {'image': tf.io.FixedLenFeature([], tf.string),
                        'label': tf.io.FixedLenFeature([], tf.string)}


    parsed_features = tf.io.parse_single_example(proto, keys_to_features)

 
    image = tf.io.parse_tensor(parsed_features['image'], out_type=tf.float32)
    image.set_shape((96, 96, 96, 1))


    label = tf.io.parse_tensor(parsed_features['label'], out_type=tf.int32)
    label.set_shape((1,))

    return image, label


def setup_input_pipeline(tfrecord_file, batch_size):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(_parse_function)
    dataset = dataset.shuffle(buffer_size=10000).batch(batch_size).prefetch(2)
    return dataset



train_dataset = setup_input_pipeline("dataset.tfrecord", batch_size=20)

In [ ]:
def create_3D_model(width, height, depth):
    inputs = tf.keras.Input((width, height, depth, 1))

    x = tf.keras.layers.Conv3D(
        filters=64, kernel_size=3, activation="relu")(inputs)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(
        filters=128, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    x = tf.keras.layers.Dense(units=256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

    model = tf.keras.models.Model(inputs, outputs)
    return model


model = create_3D_model(96, 96, 96)
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

epochs = 10
history = model.fit(train_dataset, epochs=epochs, shuffle=True)